# Chat with Model

In [1]:
import os
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient

load_dotenv(".env")

print(f"Using AZURE_AI_FOUNDRY_PROJECT_ENDPOINT: {os.environ['AZURE_AI_FOUNDRY_PROJECT_ENDPOINT']}")
print(f"Using AZURE_AI_FOUNDRY_MODEL_DEPLOYMENT_NAME: {os.environ['AZURE_AI_FOUNDRY_MODEL_DEPLOYMENT_NAME']}")

project_client = AIProjectClient(
    endpoint=os.environ["AZURE_AI_FOUNDRY_PROJECT_ENDPOINT"],
    credential=DefaultAzureCredential(),
)

openai_client = project_client.get_openai_client()

response = openai_client.responses.create(
    model=os.environ["AZURE_AI_FOUNDRY_MODEL_DEPLOYMENT_NAME"],
    input="What is the size of France in square miles?",
)
print(f"Response output: {response.output_text}")

Using AZURE_AI_FOUNDRY_PROJECT_ENDPOINT: https://agents-build2026-resource.services.ai.azure.com/api/projects/agents-build2026
Using AZURE_AI_FOUNDRY_MODEL_DEPLOYMENT_NAME: gpt-5.1
Response output: France has an area of about **248,600 square miles** (including metropolitan France in Europe but not overseas territories).


# Create Agent

In [ ]:
import os
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import PromptAgentDefinition


project_client = AIProjectClient(
    endpoint=os.environ["AZURE_AI_FOUNDRY_PROJECT_ENDPOINT"],
    credential=DefaultAzureCredential(),
)

agent = project_client.agents.create_version(
    agent_name="agents-test1",
    definition=PromptAgentDefinition(
        model=os.environ["AZURE_AI_FOUNDRY_MODEL_DEPLOYMENT_NAME"],
        instructions="You are a helpful assistant that answers general questions",
    ),
)
print(f"Agent created (id: {agent.id}, name: {agent.name}, version: {agent.version})")

Agent created (id: agents-test1:2, name: agents-test1, version: 2)


# Chat with Agent

In [ ]:
import os
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient

project_client = AIProjectClient(
    endpoint=os.environ["AZURE_AI_FOUNDRY_PROJECT_ENDPOINT"],
    credential=DefaultAzureCredential(),
)

agent_name = os.environ["AZURE_AI_FOUNDRY_AGENT_NAME"]
openai_client = project_client.get_openai_client()

# Optional Step: Create a conversation to use with the agent
conversation = openai_client.conversations.create()
print(f"Created conversation (id: {conversation.id})")

# Chat with the agent to answer questions
response = openai_client.responses.create(
    conversation=conversation.id, #Optional conversation context for multi-turn
    extra_body={"agent": {"name": agent_name, "type": "agent_reference"}},
    input="What is the size of France in square miles?",
)
print(f"Response output: {response.output_text}")

# Optional Step: Ask a follow-up question in the same conversation
response = openai_client.responses.create(
    conversation=conversation.id,
    extra_body={"agent": {"name": agent_name, "type": "agent_reference"}},
    input="And what is the capital city?",
)
print(f"Response output: {response.output_text}")

# References

https://learn.microsoft.com/en-us/azure/ai-foundry/quickstarts/get-started-code?view=foundry&tabs=python%2Cpython2

https://learn.microsoft.com/en-us/azure/ai-foundry/agents/concepts/runtime-components?view=foundry

https://learn.microsoft.com/en-us/azure/ai-foundry/agents/how-to/migrate?view=foundry